In [180]:
#import the dependencies 
import pandas as pd
import ast

In [181]:
df_unclean = pd.read_csv("./listings__20251027_061613.csv")


In [182]:
df_unclean.head()

,url,title,location,house_type,bathrooms,bedrooms,properties,amenities,price
0,https://jiji.com.gh/north-legon/houses-apartme...,1bdrm Apartment in North Legon for rent,"Greater Accra, North Legon, 3 hours ago",Apartment,1 bathroom,1 bedroom,"{'Property Address': 'North Legon', 'Property ...",NaN,"GH₵ 2,500"
1,https://jiji.com.gh/school-junction/houses-apa...,"2bdrm Apartment in Nanakrom, School Junction f...","Greater Accra, Ashaley Botwe, Ashaley Botwe Sc...",Apartment,2 bathrooms,2 bedrooms,"{'Property Address': 'School Junction', 'Estat...",NaN,"GH₵ 3,000"
2,https://jiji.com.gh/malejor/houses-apartments-...,"2bdrm Apartment in Adenta Dodowa, Malejor for ...","Greater Accra, Adenta, Malejor, 3 hours ago",Apartment,2 bathrooms,2 bedrooms,"{'Property Address': 'Major', 'Estate Name': '...",NaN,"GH₵ 2,000"
3,https://jiji.com.gh/cantonments/houses-apartme...,"4bdrm House in Cantonment, Cantonments for rent","Greater Accra, Cantonments, 3 hours ago",House,4 bathrooms,4 bedrooms,"{'Property Address': 'Cantonment', 'Estate Nam...",NaN,"GH₵ 72,000"
4,https://jiji.com.gh/american-house/houses-apar...,1bdrm Apartment in American House for rent,"Greater Accra, East Legon, American House, 3 h...",Apartment,1 bathroom,1 bedroom,"{'Property Address': 'American House', 'Proper...",NaN,"GH₵ 3,500"


In [183]:
#grab all columns
df_unclean.columns

Index(['url', 'title', 'location', 'house_type', 'bathrooms', 'bedrooms',
       'properties', 'amenities', 'price'],
      dtype='object')

In [184]:
df_unclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16356 entries, 0 to 16355
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   url         16356 non-null  object
 1   title       16356 non-null  object
 2   location    16356 non-null  object
 3   house_type  16327 non-null  object
 4   bathrooms   16353 non-null  object
 5   bedrooms    16353 non-null  object
 6   properties  16356 non-null  object
 7   amenities   13991 non-null  object
 8   price       16356 non-null  object
dtypes: object(9)
memory usage: 1.1+ MB


In [185]:
df_unclean.location.unique()

array(['Greater Accra, North Legon, 3 hours ago',
       'Greater Accra, Ashaley Botwe, Ashaley Botwe School Junction, 3 hours ago',
       'Greater Accra, Adenta, Malejor, 3 hours ago', ...,
       'Greater Accra, Ablekuma, 29/07',
       'Greater Accra, Awoshie, A-Lang / Awoshie, 03/06',
       'Greater Accra, Tema Metropolitan, 29/07'],
      shape=(4533,), dtype=object)

In [186]:
df_unclean["sub_loc"]  = df_unclean["location"].str.split(",").str.get(1).str.strip()
df_unclean["sub_loc"].unique()

array(['North Legon', 'Ashaley Botwe', 'Adenta', 'Cantonments',
       'East Legon', 'Teshie', 'Abokobi', 'Oyarifa', 'Achimota',
       'Ga West Municipal', 'Amrahia', 'Dome', 'Tantra Hills', 'Spintex',
       'Adjiriganor', 'Abelemkpe', 'Pokuase', 'McCarthy Hill', 'Agbogba',
       'West Legon', 'Old Ashomang', 'Accra Metropolitan',
       'Airport Residential Area', 'Haatso', 'Tema Metropolitan',
       'Taifa-Burkina', 'Gbawe', 'Ablekuma', 'Weija', 'Madina', 'Awoshie',
       'Nungua', 'Ofankor', 'Dansoman', 'Oyibi', 'Ga South Municipal',
       'Ashomang Estate', 'Ashaiman Municipal', 'Odorkor', 'Lapaz',
       'Kasoa', 'Osu', 'Sowutwuom', 'Dzorwulu', 'Kaneshie', 'Darkuman',
       'Mataheko', '2 hours ago', 'Kotobabi', 'Lartebiokoshie', 'Tesano',
       'Labadi', 'Anyaa', 'Labone', 'Ga East Municipal', 'Burma Camp',
       'Ridge', 'Dworwulu', '1 hour ago', 'Asylum Down', 'Danfa',
       'Santa Maria', 'North Industrial Area', 'Nima', 'Dodowa',
       'Roman Ridge', 'Kwashieman', 

In [187]:
def clean_all(data):
    #change the dictionary-like string  into a real dictionary 
    data["prop_dict"] = data["properties"].apply(ast.literal_eval)
    
    #function to work on the location
    def getSubLoc(loc):
        parts = str(loc).split(",")
        if len(parts) >= 3:
            return parts[1].strip()
        return parts[0]
    
    #Extracting some variables 
    df_unclean["sub_loc"] = df_unclean["location"].apply(getSubLoc)
    
    
    # df_unclean["sub_loc"].unique()
    data["n_bathrooms"] = data["bathrooms"].str.split(" ",expand=True)[0]
    data["n_bedroomns"] = data["bedrooms"].str.split(" ", expand=True)[0]
    data["Prices"] =data["price"].str.split(" ",expand=True)[1].str.replace(",","").astype(float)
    
    
    #Work on the amenities
    #get all the amenities in a list form
    amenities_list = df_unclean["amenities"].str.split(",", expand=True)[0].value_counts().index.to_list()
    
    #loop through all the list of amenities and create new column for each amenity
    for amenity in amenities_list:
        data[amenity] = data["amenities"].str.contains(amenity,na=False).astype(int)
    
    #drop some redundant variabbles
    data.drop(["url","title","amenities","location","bathrooms","bedrooms","properties","price"],axis=1, inplace = True)
    
clean_all(df_unclean)

In [188]:
def extract_prop(property_info):
    if isinstance(property_info, dict):
        return pd.Series({
                "Property Address":property_info.get("Property Address", None),
                "Property Size": property_info.get("Property Size", None),
                "Condition":property_info.get("Condition", None),
                "Furnishing":property_info.get("Furnishing", None),
                "Minimum Rental Period":property_info.get("Minimum Rental Period", None)
            })
    return pd.Series({
        "Property Address": None,
        "Property Size": None,
        "Condition":None,
        "Furnishing": None,
        "Minimum Rental Period":None   
    }) 

df_unclean[["Property Address","Property Size","Condition","Furnishing","Minimum Rental Period"]] = df_unclean["prop_dict"].apply(extract_prop)

In [189]:
df_unclean.drop("prop_dict",axis=1,inplace=True)

In [190]:
df_unclean.head()

,house_type,sub_loc,n_bathrooms,n_bedroomns,Prices,24-hour Electricity,Air Conditioning,Balcony,Kitchen Cabinets,Dining Area,...,Chandelier,Microwave,Wi-Fi,TV,Refrigerator,Property Address,Property Size,Condition,Furnishing,Minimum Rental Period
0,Apartment,North Legon,1,1,2500.0,0,0,0,0,0,...,0,0,0,0,0,North Legon,15,Fairly Used,Unfurnished,12 Month
1,Apartment,Ashaley Botwe,2,2,3000.0,0,0,0,0,0,...,0,0,0,0,0,School Junction,25,Newly-Built,Unfurnished,12 Month
2,Apartment,Adenta,2,2,2000.0,0,0,0,0,0,...,0,0,0,0,0,Major,100,Newly-Built,Semi-Furnished,12 Month
3,House,Cantonments,4,4,72000.0,0,0,0,0,0,...,0,0,0,0,0,Cantonment,5000,Newly-Built,Unfurnished,None
4,Apartment,East Legon,1,1,3500.0,0,0,0,0,0,...,0,0,0,0,0,American House,15,Fairly Used,Unfurnished,12 Month
